## 🔐 Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login
```

or

```bash
az login --use-device-code
```

# 👤 Simple Context Provider - Customer KYC Agent

This notebook demonstrates how to create a **Simple Context Provider** that manages customer profile information for KYC (Know Your Customer) compliance.

## Features Covered:
- Creating a custom `ContextProvider` class
- Extracting structured information from conversations
- Providing dynamic context to agents based on collected data
- Managing conversation state across multiple turns

### Industry Use Case: Customer KYC Profile Collection

Our context provider will help banking agents:
- Collect required customer identification information
- Track KYC verification status
- Enforce compliance rules (must collect info before providing services)
- Store customer profile data for the session

### ⚠️ Important Disclaimer ⚠️
> **This notebook is for educational purposes only. All customer information is simulated. In production, ensure compliance with data privacy regulations (GDPR, CCPA, etc.).**

### 🔍 How Context Providers Work

| Method | When Called | Purpose |
|--------|-------------|--------|
| `invoking()` | Before agent responds | Provide additional context/instructions |
| `invoked()` | After agent responds | Extract and store information from conversation |
| `serialize()` | When saving state | Persist data for thread continuation |

## Prerequisites

Before running this notebook, ensure you have:

1. **Microsoft Foundry Project**: With a deployed model (gpt-4o recommended)
2. **Authentication**: Azure CLI installed and authenticated
3. **Environment Variables** in root `.env` file:
   - `AI_FOUNDRY_PROJECT_ENDPOINT`
   - `AZURE_AI_MODEL_DEPLOYMENT_NAME`

If you need to use a different tenant:
```bash
az login --tenant <tenant-id>
```

## Import Libraries

Import the required libraries for creating a simple context provider:

In [ ]:
import os
from collections.abc import MutableSequence, Sequence
from typing import Any

from agent_framework import ChatAgent, ChatClientProtocol, ChatMessage, Context, ContextProvider
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from pydantic import BaseModel
from dotenv import load_dotenv

# Load environment variables from root .env
load_dotenv('../../.env', override=True)

# Verify environment setup
required_vars = [
    'AI_FOUNDRY_PROJECT_ENDPOINT',
    'AZURE_AI_MODEL_DEPLOYMENT_NAME',
]

missing = [var for var in required_vars if not os.getenv(var)]
if missing:
    print(f"⚠️ Missing environment variables: {missing}")
    print("Please configure these in your root .env file")
else:
    print("🔧 Environment Configuration:")
    print(f"✅ Project Endpoint: {os.getenv('AI_FOUNDRY_PROJECT_ENDPOINT')[:50]}...")
    print(f"✅ Model Deployment: {os.getenv('AZURE_AI_MODEL_DEPLOYMENT_NAME')}")

## Configuration 📋

Set up the configuration for Microsoft Foundry:

In [ ]:
# Microsoft Foundry configuration
PROJECT_ENDPOINT = os.environ["AI_FOUNDRY_PROJECT_ENDPOINT"]
MODEL_DEPLOYMENT = os.environ.get("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o")

print("📊 Configuration:")
print(f"  Project Endpoint: {PROJECT_ENDPOINT[:50]}...")
print(f"  Model Deployment: {MODEL_DEPLOYMENT}")

## Define Customer Profile Model 📝

Create a Pydantic model to represent customer KYC information that the context provider will collect and manage:

In [ ]:
class CustomerProfile(BaseModel):
    """Customer KYC profile information."""
    full_name: str | None = None
    account_type: str | None = None  # e.g., "checking", "savings", "investment"
    annual_income: str | None = None  # income bracket
    employment_status: str | None = None  # e.g., "employed", "self-employed", "retired"

print("✅ CustomerProfile model defined")
print("   Fields: full_name, account_type, annual_income, employment_status")

## Create the KYC Context Provider 🔍

This is the core of the notebook - a custom `ContextProvider` that:

1. **`invoking()`**: Before each agent call, provides instructions based on what customer info is still missing
2. **`invoked()`**: After each agent call, extracts customer information from the conversation
3. **`serialize()`**: Saves the customer profile for thread persistence

This pattern is useful for:
- KYC compliance workflows
- Customer onboarding
- Progressive data collection
- Personalized service delivery

In [ ]:
class CustomerKYCProvider(ContextProvider):
    """Context provider that manages customer KYC profile collection.
    
    This provider:
    - Extracts customer information from conversations
    - Provides context instructions based on what info is missing
    - Enforces KYC compliance by requiring information before service
    """
    
    def __init__(
        self, 
        chat_client: ChatClientProtocol, 
        customer_profile: CustomerProfile | None = None,
        **kwargs: Any
    ):
        """Initialize the KYC context provider.
        
        Args:
            chat_client: The chat client to use for extracting information
            customer_profile: Optional pre-populated customer profile
            **kwargs: Additional fields to populate the profile
        """
        self._chat_client = chat_client
        
        if customer_profile:
            self.customer_profile = customer_profile
        elif kwargs:
            self.customer_profile = CustomerProfile.model_validate(kwargs)
        else:
            self.customer_profile = CustomerProfile()
    
    async def invoked(
        self,
        request_messages: ChatMessage | Sequence[ChatMessage],
        response_messages: ChatMessage | Sequence[ChatMessage] | None = None,
        invoke_exception: Exception | None = None,
        **kwargs: Any,
    ) -> None:
        """Extract customer information from messages after each agent call.
        
        This method is called AFTER the agent responds. It uses the chat client
        to extract structured customer information from the conversation.
        """
        # Get user messages from the request
        user_messages = [
            msg for msg in (request_messages if isinstance(request_messages, Sequence) else [request_messages])
            if hasattr(msg, "role") and msg.role.value == "user"
        ]
        
        # Check if we still need to extract information
        needs_extraction = (
            self.customer_profile.full_name is None or
            self.customer_profile.account_type is None or
            self.customer_profile.annual_income is None or
            self.customer_profile.employment_status is None
        )
        
        if needs_extraction and user_messages:
            try:
                # Use the chat client to extract structured information
                result = await self._chat_client.get_response(
                    messages=request_messages,
                    instructions=(
                        "Extract customer KYC information from the message if present. "
                        "Look for: full name, account type (checking/savings/investment), "
                        "annual income bracket, and employment status (employed/self-employed/retired). "
                        "Return nulls for any information not provided."
                    ),
                    options={"response_format": CustomerProfile},
                )
                
                # Update profile with extracted data (only fill in missing fields)
                if result.value and isinstance(result.value, CustomerProfile):
                    if self.customer_profile.full_name is None and result.value.full_name:
                        self.customer_profile.full_name = result.value.full_name
                    if self.customer_profile.account_type is None and result.value.account_type:
                        self.customer_profile.account_type = result.value.account_type
                    if self.customer_profile.annual_income is None and result.value.annual_income:
                        self.customer_profile.annual_income = result.value.annual_income
                    if self.customer_profile.employment_status is None and result.value.employment_status:
                        self.customer_profile.employment_status = result.value.employment_status
                        
            except Exception:
                pass  # Failed to extract, continue without updating
    
    async def invoking(
        self, 
        messages: ChatMessage | MutableSequence[ChatMessage], 
        **kwargs: Any
    ) -> Context:
        """Provide customer profile context before each agent call.
        
        This method is called BEFORE the agent responds. It provides
        instructions based on what customer information is still missing.
        """
        instructions: list[str] = []
        
        # Check what information we still need
        if self.customer_profile.full_name is None:
            instructions.append(
                "The customer's name is not yet known. Politely ask for their full name "
                "before providing any account services."
            )
        else:
            instructions.append(f"The customer's name is {self.customer_profile.full_name}.")
        
        if self.customer_profile.account_type is None:
            instructions.append(
                "Ask what type of account they are interested in (checking, savings, or investment)."
            )
        else:
            instructions.append(f"The customer is interested in a {self.customer_profile.account_type} account.")
        
        if self.customer_profile.employment_status is None:
            instructions.append(
                "For KYC compliance, ask about their employment status (employed, self-employed, or retired)."
            )
        else:
            instructions.append(f"The customer's employment status is: {self.customer_profile.employment_status}.")
        
        if self.customer_profile.annual_income is None:
            instructions.append(
                "For account suitability, ask about their approximate annual income bracket."
            )
        else:
            instructions.append(f"The customer's income bracket is: {self.customer_profile.annual_income}.")
        
        # If all info collected, add completion message
        if all([
            self.customer_profile.full_name,
            self.customer_profile.account_type,
            self.customer_profile.employment_status,
            self.customer_profile.annual_income
        ]):
            instructions.append(
                "\nKYC profile is complete! You can now provide full banking services and recommendations."
            )
        
        return Context(instructions=" ".join(instructions))
    
    def serialize(self) -> str:
        """Serialize the customer profile for thread persistence."""
        return self.customer_profile.model_dump_json()
    
    def get_profile_status(self) -> dict:
        """Get the current status of the customer profile."""
        return {
            "full_name": self.customer_profile.full_name or "❌ Not provided",
            "account_type": self.customer_profile.account_type or "❌ Not provided",
            "employment_status": self.customer_profile.employment_status or "❌ Not provided",
            "annual_income": self.customer_profile.annual_income or "❌ Not provided",
            "is_complete": all([
                self.customer_profile.full_name,
                self.customer_profile.account_type,
                self.customer_profile.employment_status,
                self.customer_profile.annual_income
            ])
        }

print("✅ CustomerKYCProvider class defined")
print("   Methods: invoking(), invoked(), serialize(), get_profile_status()")

## Define Agent Instructions 🏦

Create instructions for our banking KYC agent:

In [ ]:
AGENT_INSTRUCTIONS = """
You are a professional Banking Customer Service Representative for Contoso Bank.

## Your Role:
- Help customers open new accounts and understand banking products
- Collect required KYC (Know Your Customer) information
- Provide personalized recommendations based on customer profile

## Guidelines:
1. **Be Professional**: Use a warm, professional tone
2. **Follow KYC Rules**: Collect required information before providing detailed services
3. **Address by Name**: Once you know the customer's name, use it in your responses
4. **Be Helpful**: Explain why information is needed when asked

## Required Disclaimers:
- This is for informational purposes only
- Actual account opening requires formal application and verification

## Response Style:
- Keep responses concise and friendly
- Ask one or two questions at a time, not all at once
- Acknowledge information as customers provide it
"""

print("📝 Agent Instructions Configured")
print(f"   Length: {len(AGENT_INSTRUCTIONS)} characters")

## Run the KYC Agent Demo 🚀

Now we'll demonstrate the context provider in action with a simulated customer conversation. Watch how the agent:

1. Initially asks for required information
2. Progressively collects customer profile data
3. Changes behavior once KYC is complete

In [ ]:
async def run_kyc_agent_demo():
    """Run the KYC agent demonstration with context provider."""
    
    print("=" * 60)
    print("👤 Customer KYC Agent Demo")
    print("   Using Simple Context Provider")
    print("=" * 60 + "\n")
    
    async with AzureCliCredential() as credential:
        # Create the chat client
        chat_client = AzureAIAgentClient(
            project_endpoint=PROJECT_ENDPOINT,
            AZURE_AI_MODEL_DEPLOYMENT_NAME=MODEL_DEPLOYMENT,
            credential=credential,
        )
        
        async with chat_client:
            # Create the KYC context provider
            kyc_provider = CustomerKYCProvider(chat_client)
            
            # Create the agent with the context provider
            async with ChatAgent(
                chat_client=chat_client,
                name="kyc-banking-agent",
                instructions=AGENT_INSTRUCTIONS,
                context_provider=kyc_provider,
            ) as agent:
                print(f"✅ Agent created: {agent.name}")
                print(f"🔍 Context provider: CustomerKYCProvider")
                
                # Create a new thread for the conversation
                thread = agent.get_new_thread()
                print(f"📝 Thread created for conversation\n")
                
                # Simulated customer conversation
                customer_messages = [
                    "Hi, I'd like to open a new account please.",
                    "My name is Sarah Johnson.",
                    "I'm interested in a savings account.",
                    "I work as a software engineer, so I'm employed full-time.",
                    "My annual income is around $120,000.",
                    "What savings accounts would you recommend for me?"
                ]
                
                for i, message in enumerate(customer_messages, 1):
                    print("-" * 60)
                    print(f"👤 Customer: {message}")
                    print()
                    
                    # Get agent response
                    response = await agent.run(message, thread=thread)
                    print(f"🏦 Agent: {response}")
                    print()
                    
                    # Show profile status after each turn
                    status = kyc_provider.get_profile_status()
                    print(f"📊 Profile Status:")
                    print(f"   Name: {status['full_name']}")
                    print(f"   Account Type: {status['account_type']}")
                    print(f"   Employment: {status['employment_status']}")
                    print(f"   Income: {status['annual_income']}")
                    print(f"   KYC Complete: {'✅ Yes' if status['is_complete'] else '❌ No'}")
                    print()
                
                print("=" * 60)
                print("✅ Demo complete!")
                print()
                print("📋 Final Customer Profile:")
                print(f"   {kyc_provider.serialize()}")

# Run the demo
await run_kyc_agent_demo()

## Interactive Mode 💬

Use this cell to have your own conversation with the KYC agent:

In [ ]:
# Global variables to maintain state for interactive mode
_agent = None
_thread = None
_kyc_provider = None
_credential = None
_chat_client = None

async def start_interactive_session():
    """Start an interactive KYC session."""
    global _agent, _thread, _kyc_provider, _credential, _chat_client
    
    print("🚀 Starting interactive KYC session...\n")
    
    _credential = AzureCliCredential()
    _chat_client = AzureAIAgentClient(
        project_endpoint=PROJECT_ENDPOINT,
        AZURE_AI_MODEL_DEPLOYMENT_NAME=MODEL_DEPLOYMENT,
        credential=_credential,
    )
    
    await _chat_client.__aenter__()
    
    _kyc_provider = CustomerKYCProvider(_chat_client)
    
    _agent = ChatAgent(
        chat_client=_chat_client,
        name="interactive-kyc-agent",
        instructions=AGENT_INSTRUCTIONS,
        context_provider=_kyc_provider,
    )
    
    await _agent.__aenter__()
    _thread = _agent.get_new_thread()
    
    print("✅ Interactive session ready!")
    print("   Use ask_kyc_agent('your message') to chat")
    print("   Use show_profile_status() to see collected info")
    print("   Use end_interactive_session() when done\n")

async def ask_kyc_agent(message: str):
    """Send a message to the KYC agent."""
    if _agent is None:
        print("❌ Session not started. Run start_interactive_session() first.")
        return
    
    print(f"👤 You: {message}\n")
    response = await _agent.run(message, thread=_thread)
    print(f"🏦 Agent: {response}\n")
    
    # Show profile status
    status = _kyc_provider.get_profile_status()
    if status['is_complete']:
        print("✅ KYC Profile Complete!")
    else:
        missing = [k for k, v in status.items() if v == "❌ Not provided"]
        print(f"📊 Still needed: {', '.join(missing)}")

def show_profile_status():
    """Show the current customer profile status."""
    if _kyc_provider is None:
        print("❌ Session not started.")
        return
    
    status = _kyc_provider.get_profile_status()
    print("📋 Customer Profile Status:")
    print(f"   Name: {status['full_name']}")
    print(f"   Account Type: {status['account_type']}")
    print(f"   Employment: {status['employment_status']}")
    print(f"   Income: {status['annual_income']}")
    print(f"   KYC Complete: {'✅ Yes' if status['is_complete'] else '❌ No'}")

async def end_interactive_session():
    """End the interactive session and cleanup."""
    global _agent, _thread, _kyc_provider, _credential, _chat_client
    
    if _agent:
        await _agent.__aexit__(None, None, None)
    if _chat_client:
        await _chat_client.__aexit__(None, None, None)
    if _credential:
        await _credential.__aexit__(None, None, None)
    
    _agent = None
    _thread = None
    _kyc_provider = None
    _credential = None
    _chat_client = None
    
    print("✅ Interactive session ended.")

# Start the session
await start_interactive_session()

In [ ]:
# Example interactive usage - uncomment and modify to try
await ask_kyc_agent("Hello, I want to open an account")
await ask_kyc_agent("My name is John Smith")
show_profile_status()

In [ ]:
# End the interactive session when done
await end_interactive_session()

## Key Takeaways 📚

### Creating a Simple Context Provider

```python
from agent_framework import ContextProvider, Context, ChatMessage
from pydantic import BaseModel

class MyDataModel(BaseModel):
    field1: str | None = None
    field2: str | None = None

class MyContextProvider(ContextProvider):
    def __init__(self, chat_client, **kwargs):
        self._chat_client = chat_client
        self.data = MyDataModel()
    
    async def invoking(self, messages, **kwargs) -> Context:
        # Called BEFORE agent responds
        # Return additional context/instructions
        instructions = "Additional context for the agent..."
        return Context(instructions=instructions)
    
    async def invoked(self, request_messages, response_messages, **kwargs) -> None:
        # Called AFTER agent responds
        # Extract and store information from conversation
        result = await self._chat_client.get_response(
            messages=request_messages,
            instructions="Extract info...",
            options={"response_format": MyDataModel},
        )
        if result.value:
            self.data = result.value
    
    def serialize(self) -> str:
        return self.data.model_dump_json()
```

### Using the Context Provider with an Agent

```python
# Create provider
my_provider = MyContextProvider(chat_client)

# Create agent with provider
async with ChatAgent(
    chat_client=chat_client,
    instructions="Your agent instructions",
    context_provider=my_provider,
) as agent:
    thread = agent.get_new_thread()
    response = await agent.run("User message", thread=thread)
```

### Context Provider Lifecycle

```
User Message → invoking() → Agent → invoked() → Response
                  ↓                      ↓
          Add instructions        Extract data
```

### Industry Use Cases for Context Providers

| Use Case | Data Collected | Compliance Benefit |
|----------|---------------|-------------------|
| KYC Verification | Name, ID, Address | Regulatory compliance |
| Risk Profiling | Income, Goals, Tolerance | Suitability requirements |
| Transaction Auth | Amount, Purpose, Recipient | AML compliance |
| Account Opening | Personal, Employment, Tax info | Documentation requirements |

### Environment Variables Needed

```env
# Required
AI_FOUNDRY_PROJECT_ENDPOINT=https://your-project.services.ai.azure.com/...
AZURE_AI_MODEL_DEPLOYMENT_NAME=gpt-4o
```

⚠️ **Disclaimer**: This notebook is for educational purposes. All customer scenarios are simulated. In production, ensure compliance with data privacy regulations (GDPR, CCPA, etc.) and banking regulations.